In [1]:
from model import *
m = Model()

In [2]:
def chunk():
    loss = None
    for i in range(5):
        for j in range(10):
            loss = m.train_batch(100)
        print('{}/ loss = {}'.format(i, loss))

In [3]:
chunk()

0/ loss = 0.8160468935966492
1/ loss = 0.7331170439720154
2/ loss = 0.7287883758544922
3/ loss = 0.7177261114120483
4/ loss = 0.7067042589187622


In [4]:
m.test_batch(5)

InvalidArgumentError: You must feed a value for placeholder tensor 'labels0' with dtype int32
	 [[Node: labels0 = Placeholder[dtype=DT_INT32, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op 'labels0', defined at:
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/site-packages/zmq/eventloop/io